In [7]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models

In [ ]:
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
REGION = "us-central1"
PROJECT_ID

In [8]:
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
}

In [9]:
vertexai.init(project = PROJECT_ID, location = REGION)

In [ ]:
######################################################################################
#
# generate example data
#
######################################################################################

In [3]:
UNSPSC_CODES_42294 = """Endoscopic applicators or elevators
Endoscopic aspiration or biopsy needles
Endoscopic bite blocks or straps
Endoscopic cutting instruments
Endoscopic cytology or microbiology brushes
Endoscopic clamp or dissector or grasper or forceps
Endoscopic dilators or inflation devices
Endoscopic monopolar or bipolar cable
Endoscopic hemostatic balloons or needles or tubes or accessories
Endoscopic instrument sets
Endoscopic instrument spreaders
Endoscopic insufflation needles
Endoscopic guidewire or glidewire
Endoscopic laser instruments
Endoscopic knot pushers or delivery systems
Endoscopic ligators
Endoscopic manipulators
Endoscopic monopolar or bipolar hand instruments
Endoscopic needles or punches
Endoscopic overtubes
Endoscopic instrument packs or trays or kits
Endoscopic probes
Endoscopic retractors
Endoscopic snares or snare wires
Endoscopic suction or irrigation tips or coagulation probes or ablation wand
Endoscopic suturing devices
Endoscopic working elements or working channels
Fog reduction devices for endoscopes or mirrors
Sealing caps for endoscopes
Endoscopic valves
Endoscopic biliary drainage sets
Endoscopic instrument seals
Endoscopic valve units
Endoscopic accessory kits
Endoscopic gauge
Endoscopic mouthpieces
Endoscopic guidewire handles
Endoscopic drills or drill bits
Endoscopic small joint instrument sets
Endoscopic retrievers or sets
Endoscopic extractors
Endoscopic tissue or specimen removing devices
Endoscopic hooks
Endoscopic guidewire tracers
Endoscopic/ arthroscopic shaver blades or abraders
Endoscopic vessel sealing and cutting attachments
Endoscopic sphincterotomes
Bronchoscope biopsy brushes
Bronchoscope biopsy currettes
Bronchoscope biopsy forceps
Bronchoscope brush sheath and instrumentation tubing
Bronchoscope foreign body claws
Bipolar endoscopic coagulator-cutters
Endoscopic cholangiogram kits
Endoscopic electrocautery attachments
Endoscopic electrocautery systems
Endoscopic electrocautery units
Endoscopic morsellators
Endoscopic morsellator accessories
Endoscopic or surgical light sources
Endoscopic/ laparoscopic/ arthroscopic trocar or sheath or obturator or cannula
Endoscopic laser instrument accessories
Endoscopic monopolar or bipolar hand instrument accessories
Endoscopic snare or snare wire accessories
Endoscopic suction or irrigation tip or coagulation probe accessories
Endoscopic valve accessories
Endoscopic tissue or specimen removing device accessories
Endoscopic hook accessories
Endoscopic shaver blade accessories
Endoscopic vessel sealing and cutting attachment accessories
Endoscopic electrocautery system accessories
Endoscopic or surgical light source accessories
Endoscopic suction or irrigation instrument
Endoscopic trocar or sheath or obturator or cannula accessories
Reprocessed/ sustainable Endoscopic suction or irrigation tips or coagulation probes or ablation wands
Reprocessed/ sustainable Endoscopic/ arthroscopic shaver blades or abraders
Reprocessed/ sustainable endoscopic cutting instruments
Reprocessed endoscopic/ laparoscopic/ arthroscopic trocars or sheathes or obturators or cannulae""".split("\n")

UNSPSC_CODES_42294[:5]

['Endoscopic applicators or elevators',
 'Endoscopic aspiration or biopsy needles',
 'Endoscopic bite blocks or straps',
 'Endoscopic cutting instruments',
 'Endoscopic cytology or microbiology brushes']

In [ ]:
UNSPSC_CODES_42295 = """Endoscope maintenance units
Endoscope storage cabinets
Endoscope wall hangers
Endoscopic equipment or procedure carts
Endoscopic equipment sets
Endoscopic heater probe units or heater probes
Endoscope tip protector or covers
Endoscopic instrument cases
Endoscopic lenses
Surgical or endoscopic video cable
Endoscope maintenance unit accessories
Endoscope storage cabinet accessories
Endoscope wall hanger accessories
Endoscopic equipment or procedure cart accessories
Endoscopic heater probe unit or heater probe accessories""".split("\n")

UNSPSC_CODES_42295[:5]

In [14]:
# define the LLM
model_name = "gemini-1.5-flash-001"

system_instruction = "You are an expert in supply chain and the United Nations Standard Products and Services Code (UNSPSC) to classify goods and services."

model = GenerativeModel(model_name
                        #, tools=tools
                        , system_instruction = [system_instruction]
                           )

In [18]:
instructions = """For the item provided, generate a hypothetical product description that might represent the item that is no longer than one sentence. 
Return only the text string of description.""".replace("\n", " ")

In [24]:
UNSPSC_CODES = list()

for item in UNSPSC_CODES_42294:
    prompt = f"""<INSTRUCTIONS>{instructions}</INSTRUCTIONS> \n\n <ITEM>{item}</ITEM>"""
    
    # start the chat
    chat = model.start_chat()
    r = chat.send_message([prompt]
                          , generation_config=generation_config
                          , safety_settings=safety_settings
                         )
    
    model_output = r.candidates[0].content.parts[0].text
    formatted_output = f"""<ITEM>{item}</ITEM><DESCRIPTION>{model_output.strip()}</DESCRIPTION>"""
    UNSPSC_CODES.append( formatted_output )    

In [ ]:
######################################################################################
#
# BEGIN use case
#
######################################################################################

In [30]:
PRODUCT_DATA = "\n".join(UNSPSC_CODES)
print(PRODUCT_DATA[:1000])

<ITEM>Endoscopic applicators or elevators</ITEM><DESCRIPTION>Medical devices used to introduce and manipulate instruments during endoscopic procedures.</DESCRIPTION>
<ITEM>Endoscopic aspiration or biopsy needles</ITEM><DESCRIPTION>Disposable, sterile needles for endoscopic aspiration or biopsy procedures.</DESCRIPTION>
<ITEM>Endoscopic bite blocks or straps</ITEM><DESCRIPTION>Medical devices used to stabilize a patient's mouth during endoscopic procedures.</DESCRIPTION>
<ITEM>Endoscopic cutting instruments</ITEM><DESCRIPTION>Precision surgical tools designed for minimally invasive procedures.</DESCRIPTION>
<ITEM>Endoscopic cytology or microbiology brushes</ITEM><DESCRIPTION>Disposable brushes designed for collecting cells or samples for cytology or microbiology analysis during endoscopic procedures.</DESCRIPTION>
<ITEM>Endoscopic clamp or dissector or grasper or forceps</ITEM><DESCRIPTION>A surgical instrument used for grasping, dissecting, or clamping tissue during endoscopic procedur

In [32]:
# define the LLM
model_name = "gemini-1.5-pro-001"

system_instruction = "You are an expert in supply chain and the United Nations Standard Products and Services Code (UNSPSC) to classify goods and services."

model = GenerativeModel(model_name
                        #, tools=tools
                        , system_instruction = [system_instruction]
                           )

In [33]:
instructions = """You are a medical supply expert reviewing records in the PRODUCT_DATA below. Analyze all of the PRODUCT_DATA below as a group. Your task is to:

1. Review the product description for each record.
2. Create subgroups within the PRODUCT_DATA where products of the same type (e.g., ‘endoscopic grasper’ or ‘suture passer’) are grouped together.
3. Compare product descriptions, focusing on key terms (noun and adjective) that are the same or similar (e.g., 'electrode ekg' and ‘electrocardiogram electrode’).
4. Ensure different items are not grouped together (e.g., 'clip' vs. 'clip remover').
5. Use brand names to include items in a group, but not to exclude them.
6. Note significant differences if only one description contains terms like 'refill' or 'accessory'.
7. For ambiguous cases where an item could potentially fit into multiple groups, put them in their own group.

After subgrouping, assign a distinct group number (e.g., “Group 1”) to each subgroup. For each record, provide:
- The group number
- The reason for inclusion in that group
- A confidence score (1-10) on the recommendation's support by the dataset.

Respond with each record, the group number, and detailed reasoning for the recommendation, followed by the confidence score.

"""

In [36]:
prompt = f"""<INSTRUCTIONS>{instructions}</INSTRUCTIONS> \n\n <PRODUCT_DATA>{PRODUCT_DATA}</PRODUCT_DATA> \n"""

# start the chat
chat = model.start_chat()
r = chat.send_message([prompt]
                      , generation_config=generation_config
                      , safety_settings=safety_settings
                     )

model_output = r.candidates[0].content.parts[0].text
print(model_output)

In [37]:
print(model_output)

## Analysis of PRODUCT_DATA: Endoscopic Instruments and Accessories 

Here's a breakdown of the product data, organized into groups based on similarity and purpose:

**Group 1: Endoscopic Grasping and Manipulation Instruments**

* **ITEM:** Endoscopic clamp or dissector or grasper or forceps
    * **Reason:**  This item lists several terms clearly related to grasping and manipulating tissue during endoscopic procedures.
    * **Confidence:** 10/10
* **ITEM:** Endoscopic hooks 
    * **Reason:** Hooks are designed for grasping and manipulation, aligning with the broader group purpose.
    * **Confidence:** 9/10
* **ITEM:** Endoscopic hook accessories
    * **Reason:**  Accessories designed for endoscopic hooks would support their use in grasping and manipulation.
    * **Confidence:** 8/10

**Group 2: Endoscopic Cutting and Dissecting Instruments**

* **ITEM:** Endoscopic cutting instruments
    * **Reason:** Clear focus on cutting for minimally invasive procedures.
    * **Confidence:*

In [38]:
chat_continuance = """Review the classifications decisions you made in the last step. 
Think about how the classifications could be improved given the instructions and your initial output. 
Update all of the classification results to reflect any improvements you make""".replace("\n", " ")

r = chat.send_message([chat_continuance]
                      , generation_config=generation_config
                      , safety_settings=safety_settings
                     )

print(r.candidates[0].content.parts[0].text)

## Revised Analysis of PRODUCT_DATA: Endoscopic Instruments and Accessories 

After reviewing my initial classifications, I've made several adjustments for better accuracy and consistency, considering the provided instructions: 

**Group 1: Endoscopic Grasping and Manipulation Instruments**

* **ITEM:** Endoscopic clamp or dissector or grasper or forceps
    * **Reason:**  Clear terms for tissue grasping and manipulation.
    * **Confidence:** 10/10
* **ITEM:** Endoscopic hooks 
    * **Reason:** Hooks are for grasping and manipulation. 
    * **Confidence:** 9/10
* **ITEM:** Endoscopic hook accessories
    * **Reason:**  Directly support the use of endoscopic hooks.
    * **Confidence:** 8/10
* **ITEM:** Endoscopic manipulators 
    * **Reason:**  While broad, it fits best with instruments for manipulation within the body.
    * **Confidence:** 7/10 (increased due to better group fit)

**Group 2: Endoscopic Cutting and Dissecting Instruments**

* **ITEM:** Endoscopic cutting instrumen